In [1]:
%run Preprocessing.ipynb

In [2]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from happytransformer import HappyTextClassification

In [20]:
sentimentRecord = "I hate this product!"

In [13]:
# To classify emotion
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

In [14]:
emotion_metrics = emotion_classifier(sentimentRecord)
emotion_metrics

[[{'label': 'anger', 'score': 0.1139894425868988},
  {'label': 'disgust', 'score': 0.06430400907993317},
  {'label': 'fear', 'score': 0.32141345739364624},
  {'label': 'joy', 'score': 0.09722688794136047},
  {'label': 'neutral', 'score': 0.2678925096988678},
  {'label': 'sadness', 'score': 0.09597740322351456},
  {'label': 'surprise', 'score': 0.039196278899908066}],
 [{'label': 'anger', 'score': 0.6882667541503906},
  {'label': 'disgust', 'score': 0.24790909886360168},
  {'label': 'fear', 'score': 0.005835611838847399},
  {'label': 'joy', 'score': 0.002209014492109418},
  {'label': 'neutral', 'score': 0.04385418817400932},
  {'label': 'sadness', 'score': 0.009062155149877071},
  {'label': 'surprise', 'score': 0.0028630271553993225}],
 [{'label': 'anger', 'score': 0.009969945065677166},
  {'label': 'disgust', 'score': 0.007473483681678772},
  {'label': 'fear', 'score': 0.0034124599769711494},
  {'label': 'joy', 'score': 0.02103567123413086},
  {'label': 'neutral', 'score': 0.8812554478

In [15]:
# To classify if toxic
tokenizer = AutoTokenizer.from_pretrained("martin-ha/toxic-comment-model")
toxic_model = AutoModelForSequenceClassification.from_pretrained("martin-ha/toxic-comment-model")
toxic_classifier =  TextClassificationPipeline(model=toxic_model, tokenizer=tokenizer)


In [16]:
toxic_metrics = toxic_classifier(sentimentRecord)
toxic_metrics

[{'label': 'non-toxic', 'score': 0.9989055395126343},
 {'label': 'non-toxic', 'score': 0.9147370457649231},
 {'label': 'non-toxic', 'score': 0.999015212059021},
 {'label': 'non-toxic', 'score': 0.9983499050140381}]

In [21]:
# For overall positive or negative
happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

06/01/2023 08:50:09 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [22]:
overall_metrics = happy_tc.classify_text(sentimentRecord)
overall_metrics

TextClassificationResult(label='NEGATIVE', score=0.9997503161430359)

In [23]:
sentimentRecord = process_data(sentimentRecord)
metrics = {
    "data": sentimentRecord,
    "overall": {
        "category":overall_metrics.label,
        "intensity":overall_metrics.score
    },
    "emotions" : {
        "anger" : emotion_metrics[0][0]["score"],
        "digust" : emotion_metrics[0][1]["score"],
        "fear": emotion_metrics[0][2]["score"],
        "joy": emotion_metrics[0][3]["score"],
        "neutral": emotion_metrics[0][4]["score"],
        "sadness": emotion_metrics[0][5]["score"],
        "surprise": emotion_metrics[0][6]["score"],
    },
    "toxicity" : {
        "isToxic" : toxic_metrics[0]["label"] == "toxic",
        "intensity" : toxic_metrics[0]["score"]
    }
}
metrics

{'data': ['I', 'hate', 'product', '!'],
 'overall': {'category': 'NEGATIVE', 'intensity': 0.9997503161430359},
 'emotions': {'anger': 0.1139894425868988,
  'digust': 0.06430400907993317,
  'fear': 0.32141345739364624,
  'joy': 0.09722688794136047,
  'neutral': 0.2678925096988678,
  'sadness': 0.09597740322351456,
  'surprise': 0.039196278899908066},
 'toxicity': {'isToxic': False, 'intensity': 0.9989055395126343}}